# Model Tuning

This notebook will demonstate how to automatically tune model hyperparameters for predictions accuracy.

## Set-Up
We will begin with some imports

In [1]:
import autompc as ampc
import numpy as np

Loading AutoMPC...
Finished loading AutoMPC


And we will then generate some training data using the benchmark.

In [2]:
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()

system = benchmark.system
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

## Tuning

First, we create an evaluator instance.  The evaluator determines what strategy should be used for partitioning data (i.e. holdout, cross-validation, etc.) and what scoring metric to use.  Here we use an evaluator with 10\% holdout and the RMSE metric.

In [3]:
from autompc.evaluation import HoldoutModelEvaluator

evaluator = HoldoutModelEvaluator(system, metric="rmse", holdout_prop=0.1, trajs=trajs,
                                  rng=np.random.default_rng(100))

We can now create our model tuner class and add the models we wish to consider.  Here, we will consider the SINDy and MLP models.

In [4]:
from autompc.tuning import ModelTuner
from autompc.sysid import SINDy, MLP

tuner = ModelTuner(system, evaluator)
tuner.add_model(SINDy(system))
tuner.add_model(MLP(system))

/usr/lib/python3/dist-packages/pyparsing.py:1745: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


Now, we can run the tuner to automatically search the hyperparameter space!

In [5]:
tuned_model, tune_result = tuner.run(n_iters=2, rng=np.random.default_rng(100))

Evaluating Cfg:
Configuration:
  _MLP:hidden_size_1, Value: 19
  _MLP:hidden_size_2, Value: 210
  _MLP:lr, Value: 0.025269038206003618
  _MLP:n_hidden_layers, Value: '2'
  _MLP:nonlintype, Value: 'relu'
  model, Value: 'MLP'

100%|██████████| 50/50 [02:22<00:00,  2.84s/it]
Model Score  0.10740197331640083
Evaluating Cfg:
Configuration:
  _SINDy:poly_basis, Value: 'true'
  _SINDy:poly_cross_terms, Value: 'false'
  _SINDy:poly_degree, Value: 6
  _SINDy:threshold, Value: 4.8829677609366915e-05
  _SINDy:time_mode, Value: 'continuous'
  _SINDy:trig_basis, Value: 'true'
  _SINDy:trig_freq, Value: 1
  _SINDy:trig_interaction, Value: 'false'
  model, Value: 'SINDy'



/home/william/.local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.1859e-26): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/william/.local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.1859e-26): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/william/.local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.1859e-26): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/william/.local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.1859e-26): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


Model Score  0.46276534029582395
100%|██████████| 50/50 [02:39<00:00,  3.20s/it]
